## YΣ19 Artificial Intelligence II
# Homework 2

### Iglezou Myrto - 111520170038

# Model 2

In [136]:
import pandas as pd 
from pandas import DataFrame, read_csv

In [137]:
# import gdown
# url = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view?usp=sharing'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# output = "twitterData.csv"
# gdown.download(path, output, quiet=False)
# twitterData = pd.read_csv("/content/twitterData.csv")
# df = pd.DataFrame(data=twitterData)
# df.head(5)

path = r"C:\Users\myrto\Documents\GitHub\AI2-project2\SentimentTweets.csv"
file = pd.read_csv(path,index_col=False,nrows=50000)
df = pd.DataFrame(data=file)
df.head(5)

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...


In [138]:
df['target'].replace({4: 1}, inplace = True)
X = df['text']
y = df['target']

## Data pre-processing

### Removing the scpecial characters

In [139]:
import re

def RemoveSpecialCharacters(x):
    x = re.sub(r'[^\w]+', " ", x)
    return x

In [140]:
X = X.apply(lambda x: RemoveSpecialCharacters(x) )

### Lemmatize

In [141]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

X = X.apply(lambda x: WordNetLemmatizer().lemmatize(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\myrto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [142]:
X = X.apply(lambda x: x.lower())

### Removing the upercase

In [143]:
X = X.apply(lambda x: x.lower())

## Padding

In [144]:
def pad_sentence_for_window(sentence, max_row, pad_token="<pad>"):
    padding = max_row - len(sentence)
    left = (int(padding/2) if padding%2 ==0 else int(padding/2 + 1) )
    right = int(padding/2)
    return [pad_token]*left + sentence + [pad_token]*right

In [145]:
from collections import Counter

rows = Counter([len(x) for x in X])
max_row = max(rows)

X = X.apply(lambda x: x.split())
X = X.apply(lambda x: pad_sentence_for_window(x,max_row))

In [146]:
X = X.apply(lambda x: ' '.join(x))

## Vectorizing

In [147]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split

In [148]:
def vectorizeData(X_train,X_test,vectorizer):
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    return X_train,X_test

In [149]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3,   random_state = 4, stratify = y)

In [150]:
cv = CountVectorizer()
X_train, X_test =  vectorizeData(train_X, test_X,cv)

In [151]:
y_train = torch.from_numpy(np.array(train_y)).type(torch.LongTensor)

y_test = torch.from_numpy(np.array(test_y)).type(torch.LongTensor)

In [152]:
# Transforms sparse matrix to tensor
def matrix_to_tensor(X):
    coo = X.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape
    return torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

In [153]:
train_X_tensor = matrix_to_tensor(X_train)

test_X_tensor = matrix_to_tensor(X_test)

In [154]:
train_X_tensor.shape

torch.Size([35000, 45635])

## Architecture of NN

In [155]:
class Model_2(nn.Module):
    def __init__(self,input_size,num_classes = 2,hidden_size1 = 500,hidden_size2 = 100):
        super(Model_2, self).__init__()
     
        self.fc1 = nn.Linear(input_size,hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1,hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2,num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [156]:
def train(model,X,y,optimizer,loss_fn=nn.CrossEntropyLoss(),n_epoches = 10,print_loss = True):
    
    if print_loss:
        print("Loss:\n" )

    for t in range(n_epoches):
    # step 1. Zero the gradients
        model.zero_grad()
        optimizer.zero_grad()
    # step 2. Compute the output
        y_pred = model(X)
    # step 3. Compute the loss
        loss = loss_fn(y_pred, y)
        if print_loss:
            print(t, loss.item())
    # step 4. use loss to produce gradients
        loss.backward()
    # step 5. use optimizer to take gradient step
        optimizer.step()

In [157]:
def predict(model,X):
    outputs = model(X)
    return torch.max(outputs, 1)[1]

## Training and predict

In [158]:
net = Model_2(train_X_tensor.shape[1])
learning_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [159]:
train(net,train_X_tensor,y_train,optimizer,n_epoches=20)

Loss:

0 0.6932240128517151
1 0.6961637735366821
2 0.6921349167823792
3 0.6920760869979858
4 0.6925758123397827
5 0.6918685436248779
6 0.6911228895187378
7 0.690364420413971
8 0.6897668242454529
9 0.6890678405761719
10 0.6880661845207214
11 0.6869208216667175
12 0.6855653524398804
13 0.6839935779571533
14 0.6822574138641357
15 0.6803016662597656
16 0.678013265132904
17 0.6754032969474792
18 0.6724461913108826
19 0.6690433025360107
20 0.6651734709739685
21 0.6608832478523254
22 0.6560637354850769
23 0.6507779359817505
24 0.6449641585350037
25 0.6386926174163818
26 0.6319983601570129
27 0.6248931884765625
28 0.617529571056366
29 0.6100225448608398
30 0.6019507050514221
31 0.5930136442184448
32 0.5844398736953735
33 0.5766664147377014
34 0.5690391063690186
35 0.5598843097686768
36 0.5506070256233215
37 0.543576180934906
38 0.5364182591438293
39 0.5271382927894592
40 0.5193584561347961
41 0.5135121941566467
42 0.5061048269271851
43 0.49787360429763794
44 0.49189436435699463
45 0.4866291284

In [160]:
y_pred = predict(net,test_X_tensor)

In [161]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.71      0.79      0.75      7429
           1       0.77      0.69      0.73      7571

    accuracy                           0.74     15000
   macro avg       0.74      0.74      0.74     15000
weighted avg       0.74      0.74      0.74     15000

